**En este analisis se tomarán los datos de la búsqueda bicileta, el mismo intentará analizar si se puede encontar la mejor performance en relacion precio, calidad, producto y servicio.**

In [ ]:
# Importamos los datos y los convertimos a un DataFrame.

import requests
import json

import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None

if __name__ == '__main__':
    url = 'https://api.mercadolibre.com/sites/MLA/search?q=Bicileta'
    response =requests.get(url)
    
    if response.status_code == 200:
        content = response.content
        data = json.loads(content)
        dato = pd.json_normalize(data['results'])
        df = pd.DataFrame(dato)
        print(df.head(5))

Revisamos el DataFrame y observamos el porcentaje de valores faltantes que tenemos

In [ ]:
# lista de las variables que contienen valores perdidos
vars_with_na = [var for var in df.columns if df[var].isnull().sum() > 0]

# determinar el porcentaje de valores perdidos
df[vars_with_na].isnull().mean()


**Suelto las columnas que tienen un porcentaje mayor a  0.40 de valores faltantes**

In [ ]:
df = df.drop(['sale_price', 'official_store_id', 'catalog_product_id', 
              'seller.eshop.eshop_rubro', 'address.city_id', 'seller_address.city.id',
               'differential_pricing.id'], axis=1)

df.head()


**Reviso las columnas que cuentas con valores categoricos**

In [ ]:
# Inserto en una lista los valores categoricos
cat_vars = [var for var in df.columns if df[var].dtypes == 'O']

print('Cantidad de variables categoricas: ', len(cat_vars))

# Visualizo los valores categoricos
df[cat_vars].head()


**Remplazo valores faltantes con 0**

In [ ]:
df = df.apply(lambda serie: serie.fillna(0), axis=0)


**Revisamos en una tabla de contingencia la relación que se encuentra entre price/original\_price. Podemos observar que precios ofrecen descuento en sus publicaciones**

In [ ]:
pd.crosstab(index=df['price'],
            columns=df['original_price'], margins=True)

**Ahora observamos un gráfico de barras de frecuencia relativa entre price y original\_price.**

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(16,8))

plot = (100 * df['price'].value_counts() / len(df['original_price'])).plot(
kind='bar', title='Grafico frecuencia relativa')

**Graficamos la distribución de los vendedores para poder identificar si existe mayor incidencia de un lugar a otro,**

**podemos observar que el mayor porcentaje de vendedores se encuentra en CABA seguido por provincia de Buenos Aires.**

In [ ]:
plot = df['seller_address.city.name'].value_counts().plot(kind='pie', autopct='%.2f', 
                                            figsize=(11, 11),
                                            title='original_price distribucion')

**Ahora graficamos la relacion entre ubicación y tipo de usuario (listing\_type\_i). Podemos observar que todos los usuarios gold\_pro se distribuyen practicamente todos entre CABA y lomas de zamora (gran Buenos  aires),  mientras que los usuarios gold\_special se distribuyen tanto en CABA y gran** **Buenos aires.**

In [ ]:
# Gráfico de barras de original_price según ciudad de vendedor
# Podemos ver que los usuarios gold_pro se concentran en CABA mientras que los vendedores de gold_special
# se distirbuyen entre Buenos Aires y Gran Buenos Aires.

plot = pd.crosstab(index=df['seller_address.city.name'], 
            columns=df['listing_type_id']).apply(lambda r: r/r.sum() *100,
                                              axis=1).plot(kind='bar', figsize=(11, 11))

**Analizamos y graficamos si existe una relacion según la clase de vendedor y tipo de usuario (listing\_type\_id).Podemos ver que los usuarios**

**gold\_pro tienen preponderancia absoluta en la relacion price/original\_price lo que puede traducirse en que comercializan con las bicicletas**

**de mayor valor mientras que los usuarios gold\_special tienen preponderancia en la relacion price/original\_price de gama baja y media.**

In [ ]:
plot = pd.crosstab(index=df['original_price'], 
            columns=df['listing_type_id']).apply(lambda r: r/r.sum() *100,
                                              axis=1).plot(kind='bar', figsize=(11, 11),)

**Ahora intentaremos  determinar si existe una relación entre la cantidad de ventas según su original\_price. Podemos observar que el mayor volumen se centra en bicicletas sin original\_price, mientras que la distribución de los productos que si poseen original\_price podemos identificarlos mayormente en productos de valor medio.**

In [ ]:
plot = pd.crosstab(index=df['original_price'],
            columns=df['available_quantity']
                  ).apply(lambda r: r/r.sum() *100,
                          axis=0).plot(kind='bar', stacked=True, figsize=(11, 11))

**Ahora veremos la relación entre el tipo de producto y su original\_price, podemos observar que la oferta en la bicicleta Nordic X 3.0 R29 es la única que destaca por una mayor oferta.**

In [ ]:
# Gráfico de barras de sobreviviviente segun clase
plot = pd.crosstab(index=df['title'], 
            columns=df['original_price']
                  ).apply(lambda r: r/r.sum() *100,
                          axis=0).plot(kind='bar', stacked=True, figsize=(15, 12))



**Ahora exploraremos que porcentaje de descuento tiene mayor incidencia en las ventas,  podemos observar como los valores del 35% y %50 toman notoria relevancia en comparación al resto.**

In [ ]:
df['descuento'] = ((df['price'] - df['original_price']) *100)/df['price']
df = df.round(0)

df = df.sort_values(by=['descuento'], ascending=False)

pd.crosstab(index=df['sold_quantity'],
            columns=df['descuento'], margins=True)



**Conclusión:**

**Podemos identificar a dos grupos de vendedores que predominan según su original\_price, siendo que el grupo A se focaliza en una relación precio/calidad alta (gama alta) mientras que el grupo B se centra en la relacón cantidad/precio optimo. En todos los casos el env{i es gratuito. Notamos que la ubicación que pondera es CABA siendo el mayor lugar en la oferta de original\_price, mientras que el producto más comercializado es la** **bicicleta Nordic X 3.0 R29 y los valores en los cuales original price parece ser mas efectivo son 35% (6 unidades) y 50% (7 unidades).**